In [1]:
import camelot
data = camelot.read_pdf('timetable.pdf', pages='6-25')

In [2]:
import pandas
datadf = data[0].df
for i in range(1,20):
    datadf = datadf.append(data[i].df)

In [3]:
df = datadf.copy(deep=True)
df = df.drop(index=[0]).drop(columns=[0,1]).reset_index(drop=True)
df = df.rename(columns={2:'course_code', 3:'course_name', 4:'units', 5:'ltp', 6:'sec', 7:'instructor', 8:'time', 9:'compre'})

In [4]:
df['erp_course_no'] = df['course_code'].apply(lambda x : x.split()[0])
df['course_code'] = df['course_code'].apply(lambda x : x.split()[1]+' '+x.split()[2])

In [5]:
df = df.dropna(subset=['time'])
df.iloc[pandas.Index(df['time']).get_loc('T TH 5-6:30 PM')]['time'] = 'T TH 10 11'
df.iloc[pandas.Index(df['time']).get_loc('TH S 12-1:30 PM')]['time'] = 'TH S 5 6'

In [6]:
days = {'M':-1, 'T':11, 'W':23, 'TH':35, 'F':47, 'S':59}

def reduce(code):
    codes = [code]
    for i in range(len(code)-1):
        if code[i] not in days and code[i+1] in days:
            codes = [(code[0:i+1])] + reduce(code[i+1:])
    return codes

def decode(codes):
    time = [0] * 72
    for code in codes:
        hr = []
        dy = []
        for i in code:
            if i in days:
                dy.append(i)
            else:
                hr.append(i)             
        for d in dy:
            for h in hr:
                time[days[d] + int(h)] = 1
    return time

In [7]:
df['time'] = df['time'].apply(lambda code : decode(reduce(code.split())))

In [8]:
def merge_sec(sec):
    ans = sec[['course_name','units','ltp','instructor','time','compre','erp_course_no']].iloc[0].copy(deep=True)
    for s in range(1,len(sec)):
        ans['ltp'] += sec.iloc[s]['ltp']
        ans['instructor'] += '// ' + sec.iloc[s]['instructor']
        ans['erp_course_no'] += '/' + sec.iloc[s]['erp_course_no']
        ans['time'] = [ans['time'][i] or sec.iloc[s]['time'][i] for i in range(72)]
    return ans

In [9]:
df = df.groupby(['course_code','sec']).apply(merge_sec)

In [10]:
df

,,course_name,units,ltp,instructor,time,compre,erp_course_no
course_code,sec,,,,,,,
BIO F211,1,BIOLOGICAL CHEMISTRY,3 0 3,LT,RAVIPRASAD ADURI/ Meenal Kowshik/ Rajesh Mehro...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",09/12/20 (FN),1011/1012
BIO F212,1,MICROBIOLOGY,3 1 4,LP,JUDITH MARIA BRAGANCA/ Utpal Roy/ Vijayashree ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",03/12/20 (FN),1241/1242
BIO F213,1,CELL BIOLOGY,3 0 3,LT,ANGSHUMAN SARKAR/ Indrani Talukdar// Angshuman...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",05/12/20 (AN),1013/1014
BIO F214,1,INTEGRATED BIOLOGY,3 0 3,LT,SUMIT BISWAS/ Arnab Banerjee/ Rajesh Mehrotra/...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",07/12/20 (FN),1015/1016
BIO F266,1,STUDY PROJECT,3,R,SRIKANTH MUTNURI,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",,1154
BIO F311,1,RECOMBINANT DNA TECH,3 0 3,LT,SAMIT CHATTOPADHYAY// Samit Chattopadhyay,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",14/12/20 (FN),1017/1018
BIO F312,1,PLANT PHYSIOLOGY,3 0 3,LT,SANDHYA MEHROTRA/ Kundan Kumar// Sandhya Mehro...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",01/12/20 (AN),1019/1020
BIO F313,1,ANIMAL PHYSIOLOGY,3 0 3,LT,DIBAKAR CHAKRABARTY// Dibakar Chakrabarty,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",11/12/20 (FN),1021/1022
BIO F366,1,LABORATORY PROJECT,3,R,SRIKANTH MUTNURI,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",,1155


In [34]:
df1 = df.copy(deep=True)

In [35]:
df1 = df1.reset_index(['course_code','sec'])

In [36]:
datadict = dict()
def make_dict(crs):
    
    secs = dict()
    for s in crs.index :
        sec = {
            'instructor' : crs['course_name'][s],
            'time' : crs['time'][s],
            'erp' : crs['erp_course_no'][s]
        }
        secs[crs['sec'][s]] = sec
        
    details = {
        'name' : crs.iloc[0]['course_name'],
        'units' : crs.iloc[0]['units'],
        'ltp' : crs.iloc[0]['ltp'],
        'compre' : crs.iloc[0]['compre'],
        'secnos' : len(secs),
        'sec' : secs
    }
    
    datadict[crs.iloc[0]['course_code']] = details

In [37]:
df1.groupby('course_code').apply(make_dict)

""


In [38]:
import json
with open("data2.json", 'w') as f :
    json.dump(datadict, f)